# Градиентный бустинг на деревьях

In [1]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import time
import datetime
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

## Задание 1-3
Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [ ]:
X_train = pd.read_csv(r"D:\works\features\features.csv")
X_test.fillna(0, inplace = True)
X_train.drop(['radiant_win', 'duration', 'tower_status_radiant','tower_status_dire','barracks_status_radiant',
             'barracks_status_dire'], inplace = True, axis = 1)
a = X_train.count()
for i in range (0,X_train.shape[1]-1,1):
    if (a[i]<z): print(a[i:i+1:1])

## Список неполностью заполненных значений с указанием реальной обстановки/причины:

    first_blood_time               77677   За первые 5 минут не всегда есть первое убийство

    first_blood_team               77677   За первые 5 минут не всегда есть первое убийство=> нет всегда такой команды

    first_blood_player1            77677   За первые 5 минут не всегда есть первое убийство=> нет всегда и такого игрока

    first_blood_player2            53243   За первые 5 минут не всегда есть первое убийство=> а вот помощников еще меньше

    radiant_bottle_time            81539   За первые 5 минут не куплен данный предмет рэдиантами

    radiant_courier_time           96538   За первые 5 минут не куплен данный предмет рэдиантами

    radiant_flying_courier_time    69751   За первые 5 минут не куплен данный предмет рэдиантами

    radiant_first_ward_time        95394   За первые 5 минут не куплен/установлен данный предмет рэдиантами

    dire_bottle_time               81087   За первые 5 минут не куплен данный предмет даирами

    dire_courier_time              96554   За первые 5 минут не куплен данный предмет даирами

    dire_flying_courier_time       71132   За первые 5 минут не куплен данный предмет даирами

    dire_first_ward_time           95404   За первые 5 минут не куплен данный предмет даирами


# Задание 4
Какой столбец содержит целевую переменную? Запишите его название.

Целевая переменная - 'radiant_win'

## Задание 5
Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

### Итог
Классификатор настраивается довольно долго. Время настройки растет как функция от времени. Достигнут лимит качества, так как при увеличении числа деревьев качество меняется довольно незначительно. Скорее всего качество будет расти, но вскоре будет достигнут лимит. (возможно, предел находится в районе 65-67 %)

In [2]:
X_test = pd.read_csv(r"D:\works\features/features.csv", index_col='match_id')
X_test.fillna(0, inplace = True)
y_test = X_test['radiant_win']
X_test.drop(['radiant_win', 'duration', 'tower_status_radiant','tower_status_dire','barracks_status_radiant',
             'barracks_status_dire'], inplace = True, axis = 1)
#scaler = StandardScaler()
#X_test = scaler.fit_transform(X_test)
# Создание выборки для обучения
kf = KFold(n_splits = 5, shuffle=True)
kf.split(X_test, y_test)

# Градиентный бустинг на деревьях решений
start_time = datetime.datetime.now()
GBC = GradientBoostingClassifier(n_estimators=10)#,max_depth = 3, learning_rate =0.1 )
GBC.fit(X_test, y_test)
a = cross_val_score(GBC, X_test, y_test, cv = kf, scoring = 'roc_auc')
print( 'Время для обучения 10 деревьев ', datetime.datetime.now() - start_time )
print(a.mean())

# Градиентный бустинг на деревьях решений
start_time = datetime.datetime.now()
GBC = GradientBoostingClassifier(n_estimators=20,max_depth = 3)#, learning_rate =0.1)
GBC.fit(X_test, y_test)
a = cross_val_score(GBC, X_test, y_test, cv = kf, scoring = 'roc_auc')
print( 'Время для обучения 20 деревьев ', datetime.datetime.now() - start_time )
print(a.mean())

# Градиентный бустинг на деревьях решений
start_time = datetime.datetime.now()
GBC = GradientBoostingClassifier(n_estimators=30,max_depth = 3)#, learning_rate =0.1)
GBC.fit(X_test, y_test)
a = cross_val_score(GBC, X_test, y_test, cv = kf, scoring = 'roc_auc')
print( 'Время для обучения 30 деревьев', datetime.datetime.now() - start_time )
print(a.mean())

Время для обучения 10 деревьев  0:00:58.612352
0.6639352040758942
Время для обучения 20 деревьев  0:01:48.171187
0.6819333335727756
Время для обучения 30 деревьев 0:02:49.859715
0.6893852841927302
